# Open Set Emotion Recognition

## Library Imports

In [1]:
import warnings
warnings.filterwarnings("ignore")
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from collections import Counter
import torch.nn as nn
import os
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
tqdm.pandas()
from tqdm import tqdm
import re
from collections import Counter
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchaudio
from transformers import HubertModel, HubertConfig
from sentence_transformers import SentenceTransformer
from functools import lru_cache
import utils
import torchosr
from torchosr.models import Openmax, TSoftmax
from torch.utils.tensorboard import SummaryWriter

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.manual_seed(42)
print(f"Using device: {device}")

Using device: cuda


In [2]:
ALL_LABELS = ["Neutral state","Frustration","Anger","Sadness","Happiness","Excited","Surprise","Fear","Other","Disgust"]
LABELS_TO_INT = {label: i for i, label in enumerate(ALL_LABELS)}
CLOSED_LABELS = ALL_LABELS[:5]
OTHER_LABEL = len(CLOSED_LABELS)
CREATE_AND_SAVE_AUGMENTED_FILES = False

#### Audio Features Using Hubert

In [3]:
SAVED_DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), "SAVED_DATA")
if not os.path.exists(SAVED_DATA_DIR):
    os.makedirs(SAVED_DATA_DIR)

SAVED_HUBERT_EMBEDDINGS = os.path.join(SAVED_DATA_DIR, "hubert_embeddings.pickle")

if os.path.exists(SAVED_HUBERT_EMBEDDINGS):
    hubert_embeddings = utils.load_data_using_pickle(SAVED_HUBERT_EMBEDDINGS)
else:
    hubert_embeddings = {} # audio path to embedding vector dict

HUBERT_MODEL_NAME = "facebook/hubert-large-ls960-ft"
# HUBERT_MODEL_NAME = "facebook/hubert-base-ls960"
HUBERT_MODEL_DIMENSION = 1024 # or 768
config = HubertConfig.from_pretrained(HUBERT_MODEL_NAME)
hubert_model = HubertModel.from_pretrained(HUBERT_MODEL_NAME, config=config)
sbert = SentenceTransformer('multi-qa-mpnet-base-dot-v1', device=device)

def save_hubert_embeddings():
    utils.save_data_using_pickle(hubert_embeddings, SAVED_HUBERT_EMBEDDINGS)

def get_hidden_states_from_audio_and_save(audio_path):
    if os.path.basename(audio_path) in hubert_embeddings:
        return hubert_embeddings[os.path.basename(audio_path)]
    
    waveform, sample_rate = torchaudio.load(audio_path)

    # Resample if necessary (HuBERT uses 16 kHz sample rate)
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)
        sample_rate = 16000

    # Ensure single channel audio (mono)
    if waveform.shape[0] > 1:
        waveform = torch.mean(waveform, dim=0, keepdim=True)

    # Normalize audio
    waveform = waveform / torch.max(torch.abs(waveform))

    # Step 3: Encode audio waveforms using HuBERT model
    # Pass the waveform through the model
    with torch.no_grad():
        outputs = hubert_model(waveform)

    # Get the hidden states
    hidden_states = outputs.last_hidden_state
    hubert_embeddings[os.path.basename(audio_path)] = hidden_states
    return hidden_states

def get_audio_features(audio_paths):
    audio_embeddings_list = []
    for audio_path in tqdm(audio_paths):
        hidden_states = get_hidden_states_from_audio_and_save(audio_path)
        frame_averaged_hidden_states = torch.mean(hidden_states, dim=1)
        frame_averaged_hidden_states = frame_averaged_hidden_states.view(-1,HUBERT_MODEL_DIMENSION)[0]
        audio_embeddings_list.append(frame_averaged_hidden_states)
    # save_hubert_embeddings()
    return audio_embeddings_list

Some weights of HubertModel were not initialized from the model checkpoint at facebook/hubert-large-ls960-ft and are newly initialized: ['hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Dataset Creation

### MELD

In [4]:
# class MELDDataset(Dataset):
#     def __init__(self, meld_dir, split, transform=None):
#         train_df = pd.read_csv("../MELD_Dataset/train_sent_emo.csv")
#         labels = train_df['Emotion'].unique().tolist()
#         self.label_to_int = {label: i for i, label in enumerate(labels)}

#         self.meld_dir = meld_dir
#         self.transform = transform
#         self.split = split
#         self.img_path = os.path.join(self.meld_dir, 'mel_spectrograms', f'{self.split}_img')
#         self.img_path = os.path.join(self.meld_dir, 'log_spectrogram', f'{self.split}_audio')

#         # load and create sentence embeddings
#         self.dialogues = self.load_dialogues()
#         sentences = self.dialogues['Utterance'].tolist()
#         sentences = [text.replace("\x92", "'") for text in sentences]
#         self.sentence_embeddings = sbert.encode(sentences, convert_to_tensor=True, show_progress_bar=True, batch_size=128, device=device)

#         self.spectrograms = self.load_spectrograms()
#         self.resnet_model = models.resnet50(pretrained=True)
#         self.feature_extractor = torch.nn.Sequential(*list(self.resnet_model.children())[:-1]).to(device)
#         self.feature_extractor.eval()

#     def load_dialogues(self):
#         dialogue_file = os.path.join(self.meld_dir, f'{self.split}_sent_emo.csv')
#         dialogues = pd.read_csv(dialogue_file)
#         return dialogues

#     def load_spectrograms(self):
#         images = os.listdir(self.img_path)
#         return images

#     def __len__(self):
#         assert(len(self.sentence_embeddings) == len(self.spectrograms))
#         return len(self.dialogues)

#     def preprocess_img(self, img):
#         preprocessor = transforms.Compose([
#             transforms.Resize(256),
#             transforms.CenterCrop(224),
#             transforms.ToTensor(),
#         ])
#         img_t =  preprocessor(img).to(device)
#         return img_t

#     def extract_audio_features_from_spectrogram(self, img):
#         # Pass the input through the model
#         with torch.no_grad():
#             output = self.feature_extractor(img)
#         return output

#     def __getitem__(self, idx):
#         row = self.dialogues.iloc[idx]
#         text = self.sentence_embeddings[idx]
#         spectrogram_data = Image.open(os.path.join(self.img_path, f'dia{row["Dialogue_ID"]}_utt{row["Utterance_ID"]}.png'))
#         spectrogram_data = self.preprocess_img(spectrogram_data)
#         spectrogram_data = spectrogram_data[0:3, :, :]
#         spectrogram_data = spectrogram_data.unsqueeze(0)
#         spectrogram_data = self.extract_audio_features_from_spectrogram(spectrogram_data)
#         spectrogram_data = spectrogram_data.view(-1, 2048)[0]
#         label = row['Emotion']
#         label = torch.tensor(self.label_to_int[label])
#         return text, spectrogram_data, label

# train_meld = MELDDataset("../MELD_Dataset", "train")
# # test_meld = MELDDataset("../MELD_Dataset", "test")
# # dev_meld = MELDDataset("../MELD_Dataset", "dev")

# # concat all 3 datasets into 1 dataset
# meld_dataset = train_meld # + test_meld + dev_meld

In [5]:
# len(meld_dataset)

#### IEMOCAP

In [6]:
IEMOCAP_FULL_PATH = os.path.join(os.path.dirname(os.getcwd()),"IEMOCAP_full_release")
IEMOCAP_MAIN_FOLDER = os.path.join(IEMOCAP_FULL_PATH,"IEMOCAP_full_release")
TRANSCRIPTION_FOLDER = os.path.join("dialog", "transcriptions")
AUDIO_FOLDER = os.path.join("sentences", "wav")
CATEGORICAL_LABELS_PATH = os.path.join("dialog", "EmoEvaluation", "Categorical")
AUGMENTED_AUDIO_FOLDER = os.path.join(os.path.dirname(os.getcwd()), "augmented")

In [7]:
def get_evaluator_filenames_with_video_file_prefix(input_list, prefix_value):
    regex_pattern = re.compile(f'^{re.escape(prefix_value)}.*\.txt$')
    matching_strings = [s for s in input_list if regex_pattern.match(s)]
    return matching_strings

def get_agreed_upon_evaluation(evaluations):
    top_two_frequent_evaluations = Counter(evaluations).most_common(2)
    if(len(top_two_frequent_evaluations)==1):
        return top_two_frequent_evaluations[0][0]
    most_frequent_evaluation_and_its_frequency, second_most_frequent_evaluation_and_its_frequency = top_two_frequent_evaluations
    highest_frequency = most_frequent_evaluation_and_its_frequency[1]
    second_highest_frequency = second_most_frequent_evaluation_and_its_frequency[1]
    if(highest_frequency==second_highest_frequency):
        return "AMBIGUOUS"
    else:
        return most_frequent_evaluation_and_its_frequency[0]

def get_utterance_to_evaluationCounter_mapping_from_evaluation_files(evaluation_files):
    utterance_to_all_evaluations = {}

    for evaluation_file in evaluation_files:
        utterance_to_evaluationList = {}
        with open(evaluation_file,'r') as f:
            contents = f.read()
            utterance_evaluations = contents.split("\n")
            for evaluation in utterance_evaluations:
                evaluation = evaluation.strip()
                if(len(evaluation)==0):
                    continue
                matches = re.findall(r':[^;]+;', evaluation)
                matches = [match[1:-1] for match in matches]
                utterance_to_evaluationList[evaluation.split()[0]] = matches
        
        # Combine lists from dict1
        for key, value_list in utterance_to_evaluationList.items():
            utterance_to_all_evaluations[key] = utterance_to_all_evaluations.get(key, []) + value_list

    utterance_to_evaluationsCounter = {k: get_agreed_upon_evaluation(v) for k,v in utterance_to_all_evaluations.items()}
    return utterance_to_evaluationsCounter

def is_label_a_closed_label(evaluation):
    return evaluation in CLOSED_LABELS

def create_unprocessed_dataset(is_closed_label_set_flag):
    dataset = []
    audio_files = []
    sentences_list = []
    for session_num in range(1,6):
        for transcription_filename in os.listdir(os.path.join(IEMOCAP_MAIN_FOLDER,f"Session{session_num}", TRANSCRIPTION_FOLDER)):
            if(transcription_filename[0]!="."): 

                filename_without_extension = transcription_filename.split(".")[0]
                
                categorical_labels_folder_full_path = os.path.join(IEMOCAP_MAIN_FOLDER, f"Session{session_num}", CATEGORICAL_LABELS_PATH)
                evaluation_filenames = get_evaluator_filenames_with_video_file_prefix(os.listdir(categorical_labels_folder_full_path), filename_without_extension)
                evaluation_files_full_paths_for_this_file = [os.path.join(IEMOCAP_MAIN_FOLDER, f"Session{session_num}", CATEGORICAL_LABELS_PATH, f) for f in evaluation_filenames]
                evaluations_per_utterance = get_utterance_to_evaluationCounter_mapping_from_evaluation_files(evaluation_files_full_paths_for_this_file)
                
                transcription_file_full_path = os.path.join(IEMOCAP_MAIN_FOLDER, f"Session{session_num}", TRANSCRIPTION_FOLDER, transcription_filename) 
                with open(transcription_file_full_path,'r') as f:
                    contents = f.read()
                    lines = contents.split("\n")

                    # Iterate through utterances where every utterance looks like:
                    # Ses01F_impro01_F000 [006.2901-008.2357]: Excuse me.
                    for line in lines:

                        # Remove extra spaces and check if the line is not an empty link (usually at EOF)
                        line = line.strip()
                        if(len(line)==0):
                            break

                        # Remove idx of first space, ], -
                        try:
                            space_idx = line.index(" ")
                            timestampEndBracket_idx = line.index("]")
                            timestampHyphen_idx = line.index("-")
                        except:
                            continue
                        else:
                            audio_filename = line[:space_idx]        # output audio file name = utterance name
                            text = line[timestampEndBracket_idx+3:]         # the transcription of the utterance
                            evaluation = evaluations_per_utterance.get(audio_filename,"KEY_ERROR")

                            utterance_audios_per_video_folder = audio_filename[:line.rindex('_')]       # Only need Ses01F_impro01 from Ses01F_impro01_F000
                            audio_file_full_path = os.path.join(IEMOCAP_MAIN_FOLDER, f"Session{session_num}", AUDIO_FOLDER, utterance_audios_per_video_folder, audio_filename+".wav")         # name of the video file

                            if(evaluation!="KEY_ERROR" and evaluation!= "AMBIGUOUS" and os.path.isfile(audio_file_full_path)==True and is_label_a_closed_label(evaluation)==is_closed_label_set_flag):
                            # if(evaluation!="KEY_ERROR" and os.path.isfile(audio_file_full_path)==True):    
                                audio_files.append(audio_file_full_path)
                                sentences_list.append(text)
                                dataset.append((text,audio_file_full_path,evaluation))
    return dataset, sentences_list, audio_files

In [8]:
openSetUnprocessedDataset, openSetSentencesList, openSetAudioFiles = create_unprocessed_dataset(is_closed_label_set_flag = False)
closedSetUnprocessedDataset, closedSetSentencesList, closedSetAudioFiles= create_unprocessed_dataset(is_closed_label_set_flag = True)

In [9]:
closedSetUnprocessedDataset[5]

('Who told you to get in this line?',
 'd:\\Projects\\open-set-emotion-recognition\\IEMOCAP_full_release\\IEMOCAP_full_release\\Session1\\sentences\\wav\\Ses01F_impro01\\Ses01F_impro01_M002.wav',
 'Frustration')

In [10]:
class IemocapDataset(Dataset):
    def __init__(self, unprocessed_dataset, sentences_list, audio_files, split, is_closed_label_set_flag, save_augmented_files ) -> None:
        self.unprocessed_dataset = unprocessed_dataset
        self.is_closed_label_set_flag = is_closed_label_set_flag
        self.split = split
        self.sentence_embeddings = sbert.encode(sentences_list, convert_to_tensor=True, show_progress_bar=True, batch_size=128, device=device)

        self.save_augmented_files = save_augmented_files
        self.augment_times = 4
        # we augment just for train, so we are repeating the sentences as only audios are augmented
        # if self.split == "train":
        #     self.sentence_embeddings = np.repeat(self.sentence_embeddings, repeats=self.augment_times, axis=0)
        #     self.unprocessed_dataset = self.create_augmented_dataset()

        all_audio_files = [instance[1] for instance in self.unprocessed_dataset]
        assert(all([os.path.exists(audio_file) for audio_file in all_audio_files]), "Some audio files do not exist")
        self.audio_embeddings = get_audio_features(all_audio_files)


    def create_augmented_dataset(self):
        augmented_dataset = []
        os.makedirs(AUGMENTED_AUDIO_FOLDER, exist_ok=True)
        for text, audio_file_full_path, evaluation in self.unprocessed_dataset:
            _, audio_filename_and_extension = os.path.split(audio_file_full_path)
            audio_filename, extension = audio_filename_and_extension.split(".")
            for augmented_version_num in range(self.augment_times):
                augmented_audio_path = os.path.join(AUGMENTED_AUDIO_FOLDER,f"{audio_filename}_version_{augmented_version_num}.{extension}")
                if self.save_augmented_files:
                    utils.augment_audio_and_save(input_audio_path = audio_file_full_path, augmented_audio_path = augmented_audio_path)
                else:
                    assert(os.path.exists(augmented_audio_path), f"Augmented Audio Path {augmented_audio_path} not exist")
                augmented_dataset.append((text, augmented_audio_path, evaluation))
        return augmented_dataset
    
    def __len__(self):
        return len(self.unprocessed_dataset)
    
    def __getitem__(self, idx):
        _, audio, label = self.unprocessed_dataset[idx]
        local_device = "cuda"
        if(self.split in ["val","test"]):
            local_device = "cpu"
        text = self.sentence_embeddings[idx].to(local_device)
        audio = self.audio_embeddings[idx].to(local_device)
        combined = torch.cat([audio, text])
        if self.is_closed_label_set_flag==False:
            label = OTHER_LABEL
        else:
            label = LABELS_TO_INT[label]

        label_vec = np.zeros(len(CLOSED_LABELS)+1)
        label_vec[label] = 1
        label_vec = torch.tensor(label_vec).to(local_device)
        
        return combined, label_vec

In [11]:
def get_stratified_split(unprocessed_dataset, sentences_list, audio_files, test_size):
    # instance[-1] is label
    labels = [instance[-1] for instance in unprocessed_dataset]
    unprocessed_train_dataset, unprocessed_test_dataset, train_sentences_list, test_sentences_list, train_audio_files, test_audio_files = train_test_split(unprocessed_dataset, sentences_list, audio_files, test_size=test_size, stratify=labels, random_state=42)
    return unprocessed_train_dataset, unprocessed_test_dataset, train_sentences_list, test_sentences_list, train_audio_files, test_audio_files

In [12]:
closed_set_unprocessed_train_dataset, temp_unprocessed_dataset, closed_set_train_sentences_list, temp_sentences_list, closed_set_train_audio_files, temp_audio_files = get_stratified_split(closedSetUnprocessedDataset, closedSetSentencesList, closedSetAudioFiles, 0.2)
closed_set_unprocessed_val_dataset, closed_set_unprocessed_test_dataset, closed_set_val_sentences_list, closed_set_test_sentences_list, closed_set_val_audio_files, closed_set_test_audio_files = get_stratified_split(temp_unprocessed_dataset, temp_sentences_list, temp_audio_files, 0.5)
open_set_unprocessed_val_dataset, open_set_unprocessed_test_dataset, open_set_val_sentences_list, open_set_test_sentences_list, open_set_val_audio_files, open_set_test_audio_files = get_stratified_split(openSetUnprocessedDataset, openSetSentencesList, openSetAudioFiles, 0.5)

In [13]:
closed_set_train_dataset = IemocapDataset(unprocessed_dataset = closed_set_unprocessed_train_dataset,
                                          sentences_list = closed_set_train_sentences_list,
                                          audio_files = closed_set_train_audio_files,
                                          split = "train",
                                          is_closed_label_set_flag = True,
                                          save_augmented_files=CREATE_AND_SAVE_AUGMENTED_FILES)

closed_set_val_dataset = IemocapDataset(unprocessed_dataset = closed_set_unprocessed_val_dataset,
                                        sentences_list = closed_set_val_sentences_list,
                                        audio_files = closed_set_val_audio_files,
                                        split="val",
                                        is_closed_label_set_flag=True,
                                        save_augmented_files=CREATE_AND_SAVE_AUGMENTED_FILES)

closed_set_test_dataset = IemocapDataset(unprocessed_dataset = closed_set_unprocessed_test_dataset,
                                        sentences_list = closed_set_test_sentences_list,
                                        audio_files = closed_set_test_audio_files,
                                        split="test",
                                        is_closed_label_set_flag=True,
                                        save_augmented_files=CREATE_AND_SAVE_AUGMENTED_FILES)

open_set_val_dataset = IemocapDataset(unprocessed_dataset = open_set_unprocessed_val_dataset,
                                      sentences_list = open_set_val_sentences_list,
                                      audio_files = open_set_val_audio_files,
                                      split="val",
                                      is_closed_label_set_flag=False,
                                      save_augmented_files=CREATE_AND_SAVE_AUGMENTED_FILES)

open_set_test_dataset = IemocapDataset(unprocessed_dataset = open_set_unprocessed_test_dataset,
                                      sentences_list = open_set_test_sentences_list,
                                      audio_files = open_set_test_audio_files,
                                      split="test",
                                      is_closed_label_set_flag=False,
                                      save_augmented_files=CREATE_AND_SAVE_AUGMENTED_FILES)

Batches:   0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 597/597 [03:17<00:00,  3.03it/s]


In [14]:
print(len(closed_set_train_dataset))
print(len(closed_set_val_dataset))
print(len(closed_set_test_dataset))
print(len(open_set_val_dataset))
print(len(open_set_test_dataset))


5071
634
634
596
597


In [15]:
entire_val_dataset = closed_set_val_dataset + open_set_val_dataset
entire_test_dataset = closed_set_test_dataset + open_set_test_dataset

In [16]:
len(entire_val_dataset), len(entire_test_dataset)

(1230, 1231)

In [17]:
# Create data loaders.
batch_size = 64

# AUDIO + TEXT DATALOADERS (CONSISTS OF BOTH OPEN AND CLOSED LABELS)
train_dataloader = DataLoader(closed_set_train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(entire_val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(entire_test_dataset, batch_size=batch_size, shuffle=True)

# AUDIO + TEXT DATALOADERS (ONLY CLOSED LABELS - USED FOR CHECKING MODEL PERFORMANCE AFTER KEEPING ASIDE THE OPEN SET CHALLENGE)
val_closed_set_dataloader = DataLoader(closed_set_val_dataset, batch_size=batch_size, shuffle=True)
test_closed_set_dataloader = DataLoader(closed_set_test_dataset, batch_size=batch_size, shuffle=True)



## Model Architectures Definition

In [18]:
# MODELS PATHS
MODELS_DIR = os.path.join(os.path.dirname(os.getcwd()), "MODELS")
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

MULTIMODAL_MODEL_PATH = os.path.join(MODELS_DIR, "multimodal_model_augmented.pt")
NEW_MULTIMODAL_MODEL_PATH = os.path.join(MODELS_DIR, "multimodal_model_without_happiness.pt")
MULTIMODAL_MODEL_OPENMAX_PATH = os.path.join(MODELS_DIR, "multimodal_model_openmax.pt")
MULTIMODAL_MODEL_TSOFTMAX_PATH = os.path.join(MODELS_DIR, "multimodal_model_tsoftmax.pt")
MULTIMODAL_MODEL_OVERLAYSOFTMAX_PATH = os.path.join(MODELS_DIR, "multimodal_model_overlaysoftmax.pt")

In [19]:
class AudioTextEmotionModel(nn.Module):
    def __init__(self, num_classes):
        super(AudioTextEmotionModel, self).__init__()
        # sequential model with 2 layers, followed by dropout and relu layers and output layer
        self.fc = nn.Sequential(
            nn.Linear(HUBERT_MODEL_DIMENSION + 768, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )


    def forward(self, text_audio_combined):
        return self.fc(text_audio_combined)

## Model Training

In [20]:
class EarlyStopping:
    def __init__(self, patience=5, delta=0, path='model_checkpoint.pt'):
        self.patience = patience
        self.delta = delta
        self.path = path
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_acc_max = np.Inf

    def __call__(self, val_acc, model):

        if self.best_score is None:
            self.best_score = val_acc
            self.save_checkpoint(val_acc, model)
        elif val_acc < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = val_acc
            self.save_checkpoint(val_acc, model)
            self.counter = 0

    def save_checkpoint(self, val_acc, model):
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_acc

In [21]:
def accuracy(dataloader, model):
    size = len(dataloader.dataset)
    total_correct = 0
    model.eval()
    for batch, x_and_y in enumerate(dataloader):
        x_and_y_device = [x_and_y[i].to(device) for i in range(len(x_and_y))]

        # Compute prediction error
        pred = model(*x_and_y_device[:-1])
        predicted = torch.argmax(pred,dim=1).cpu()
        _,label = torch.max(x_and_y_device[-1],1)
        actual = label.cpu()
        correct = predicted == actual
        total_correct += correct.sum().item()
    return total_correct/size

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, x_and_y in enumerate(dataloader):
        x_and_y_device = [x_and_y[i].to(device) for i in range(len(x_and_y))]

        # Compute prediction error
        pred = model(*x_and_y_device[:-1])
        label = x_and_y_device[-1]
        _, label = torch.max(label,1)
        loss = loss_fn(pred, label)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 10 == 0:
            loss, current = loss.item(), (batch + 1) * len(x_and_y_device[0])
            
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [32]:
model = AudioTextEmotionModel(OTHER_LABEL)
model.to(device)

# TEXT + AUDIO
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=1e-3, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=3, verbose=True)
num_epochs = 100

writer = SummaryWriter()

early_stopping = EarlyStopping(patience=9, delta=0, path=NEW_MULTIMODAL_MODEL_PATH)
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    train_closed_set_accuracy = accuracy(train_dataloader,model)
    val_closed_set_accuracy = accuracy(val_closed_set_dataloader,model)
    print(f"Accuracy on Train Set => {train_closed_set_accuracy} | Accuracy on Closed Validation Set => {val_closed_set_accuracy}")
    
    writer.add_scalar('Accuracy/train', train_closed_set_accuracy, epoch)
    writer.add_scalar('Accuracy/val', val_closed_set_accuracy, epoch)

    scheduler.step(val_closed_set_accuracy)
    early_stopping(val_closed_set_accuracy,model)
    if early_stopping.early_stop:
        print("Early stopping")
        break

print(f"MODEL TRAINED...RE-LOADING BEST MODEL")
model = AudioTextEmotionModel(OTHER_LABEL)
model.to(device)
model.load_state_dict(torch.load(NEW_MULTIMODAL_MODEL_PATH))
best_multimodal_model_val_accuracy = accuracy(val_closed_set_dataloader,model)
print(f"Best Multimodal Model's accuracy on Closed Validation Set => {best_multimodal_model_val_accuracy}")

Epoch 1
-------------------------------
loss: 1.695176  [   64/ 5071]
loss: 2.247442  [  704/ 5071]
loss: 1.943730  [ 1344/ 5071]
loss: 1.626241  [ 1984/ 5071]
loss: 1.495362  [ 2624/ 5071]
loss: 1.358742  [ 3264/ 5071]
loss: 1.406313  [ 3904/ 5071]
loss: 1.416525  [ 4544/ 5071]
Accuracy on Train Set => 0.5365805561033327 | Accuracy on Closed Validation Set => 0.4637223974763407
Epoch 2
-------------------------------
loss: 1.259302  [   64/ 5071]
loss: 1.000859  [  704/ 5071]
loss: 1.063228  [ 1344/ 5071]
loss: 1.123242  [ 1984/ 5071]
loss: 0.964247  [ 2624/ 5071]
loss: 1.020280  [ 3264/ 5071]
loss: 0.823116  [ 3904/ 5071]
loss: 0.981347  [ 4544/ 5071]
Accuracy on Train Set => 0.6716623940051272 | Accuracy on Closed Validation Set => 0.5599369085173501
Epoch 3
-------------------------------
loss: 0.939890  [   64/ 5071]
loss: 0.767573  [  704/ 5071]
loss: 0.983994  [ 1344/ 5071]
loss: 0.759498  [ 1984/ 5071]
loss: 0.940905  [ 2624/ 5071]
loss: 0.899201  [ 3264/ 5071]
loss: 0.965237  

## MONTE CARLO DROPOUT

In [33]:
def set_dropout_to_train(eval_model):
    for module in eval_model.modules():
        if isinstance(module, nn.Dropout):
            module.train()

def predict(label, model, combined_data, n_simulations=100, threshold=1, other_label=OTHER_LABEL):
    predictions = [model(combined_data).detach().cpu() for _ in range(n_simulations)]
    predictions = torch.stack(predictions)
    predictions = F.softmax(predictions, dim=2)

    mean_predictions = torch.mean(predictions,dim=0)
    std_predictions = torch.mean(torch.std(predictions,dim=0),dim=1)
    _,predicted_class = torch.max(mean_predictions,1)
    high_uncertainty = std_predictions>threshold
    predicted_class[high_uncertainty]=other_label
    return predicted_class

def evaluate(model, dataloader, device, threshold=0.6):
    # After setting the model to evaluation mode, call this function
    model.eval()
    set_dropout_to_train(model)

    size = len(dataloader.dataset)
    total_correct = 0
    total_confusion_matrix = torch.zeros((OTHER_LABEL+1,OTHER_LABEL+1))
    # total_correct_pred_of_other_label, total_actual_other_label = 0,0
    for batch, (combined_data, label) in enumerate(dataloader):
        combined_data, label = combined_data.to(device), label.to(device)

        predicted = predict(label, model, combined_data, threshold=threshold)
        predicted = predicted.cpu()
        _, actual = torch.max(label.cpu(),1)
        correct = predicted == actual
        total_correct += correct.sum().item()
        cm = confusion_matrix(predicted,actual)
        if(cm.shape[0]!=(OTHER_LABEL+1) and cm.shape[1]!=(OTHER_LABEL+1)):
            row_of_zeros = np.zeros((OTHER_LABEL+1-cm.shape[0],cm.shape[1]))
            array_with_row = np.concatenate((cm, row_of_zeros), axis=0)

            # Add a column of zeros at the end
            column_of_zeros = np.zeros((OTHER_LABEL+1, OTHER_LABEL+1-cm.shape[1]))
            array_with_row_and_column = np.concatenate((array_with_row, column_of_zeros), axis=1)
            cm = array_with_row_and_column

        total_confusion_matrix+= cm
    print(f"Confusion Matrix:")
    print(total_confusion_matrix)
    return total_correct/size

In [34]:
mc_dropout_threshold = 0.10
print("Validation Set:")
print(f"Accuracy: {evaluate(model,val_dataloader,device,threshold=mc_dropout_threshold)}")
print(f"----------------------------------------------------------------")
print("Test Set:")
print(f"Accuracy: {evaluate(model,test_dataloader,device,threshold=mc_dropout_threshold)}")

Validation Set:
Confusion Matrix:
tensor([[ 92.,  22.,   9.,  14.,  11., 111.],
        [ 24., 102.,  26.,  13.,   1.,  71.],
        [  5.,  24.,  56.,   0.,   0.,  45.],
        [ 10.,   9.,   0.,  68.,   2.,   5.],
        [  6.,   2.,   1.,   1.,  24., 185.],
        [ 34.,  26.,  18.,  13.,  21., 179.]], dtype=torch.float64)
Accuracy: 0.42357723577235773
----------------------------------------------------------------
Test Set:
Confusion Matrix:
tensor([[ 94.,  25.,   5.,   7.,  12., 108.],
        [ 20.,  90.,  24.,   7.,   2.,  67.],
        [  7.,  24.,  66.,   0.,   1.,  42.],
        [  7.,   5.,   0.,  78.,   2.,  14.],
        [  6.,   3.,   1.,   0.,  30., 171.],
        [ 37.,  38.,  14.,  16.,  13., 195.]], dtype=torch.float64)
Accuracy: 0.44922826969943136


## TEMPERATURE SCALING

In [25]:
def predict_temp(model, combined_data,threshold=1, temperature=1.0):
    with torch.no_grad():
        predictions = model(combined_data)
        
        # Apply temperature scaling
        predictions = predictions/temperature
        
        probabilities = F.softmax(predictions, dim=1)
        max_val, predicted_class = torch.max(probabilities, 1)

        predicted_class[max_val<threshold] = OTHER_LABEL

        return predicted_class

def evaluate_temp(model, dataloader, device, threshold, temperature):
    model.eval()
    size = len(dataloader.dataset)
    total_confusion_matrix = torch.zeros((OTHER_LABEL+1,OTHER_LABEL+1))

    total_correct = 0
    for combined_data, label in dataloader:
        combined_data, label = combined_data.to(device), label.to(device)

        predicted = predict_temp(model, combined_data, threshold, temperature)
        predicted = predicted.cpu()
        _, actual = torch.max(label.cpu(),1)
        correct = (predicted == actual)
        total_correct += correct.sum().item()
        cm = confusion_matrix(predicted,actual)
        if(cm.shape[0]!=(OTHER_LABEL+1) and cm.shape[1]!=(OTHER_LABEL+1)):
            row_of_zeros = np.zeros((OTHER_LABEL+1-cm.shape[0],cm.shape[1]))
            array_with_row = np.concatenate((cm, row_of_zeros), axis=0)
            column_of_zeros = np.zeros((OTHER_LABEL+1, OTHER_LABEL+1-cm.shape[1]))
            array_with_row_and_column = np.concatenate((array_with_row, column_of_zeros), axis=1)
            cm = array_with_row_and_column

        total_confusion_matrix+= cm

    return total_correct/size, total_confusion_matrix

In [26]:
def select_best(model, dataloader, device, thresholds, temperatures):
    best_accuracy = 0
    best_params = {'threshold': 0, 'temperature': 0}
    results = []

    for threshold in thresholds:
        for temperature in temperatures:
            accuracy, confusion_matrix = evaluate_temp(model, dataloader, device, threshold, temperature)
            results.append((threshold, temperature, accuracy))
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['threshold'] = threshold
                best_params['temperature'] = temperature
    
    return best_params, best_accuracy, results

thresholds = np.linspace(0.1, 2, 20)  
temperatures = np.linspace(0.5, 2, 16)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

best_params, best_accuracy, results = select_best(model, val_dataloader, device, thresholds, temperatures)
print(f'Best Parameters: {best_params} with an accuracy of {best_accuracy}')

Best Parameters: {'threshold': 0.7999999999999999, 'temperature': 1.9000000000000001} with an accuracy of 0.4983739837398374


In [35]:
evaluate_temp(model, test_dataloader, device, best_params["threshold"], best_params["temperature"])

(0.4703493095044679,
 tensor([[ 56.,  10.,   3.,   4.,   9.,  52.],
         [  5.,  45.,   8.,   3.,   2.,  24.],
         [  2.,  12.,  48.,   0.,   1.,  26.],
         [  2.,   0.,   0.,  59.,   4.,   3.],
         [  3.,   3.,   2.,   0.,  27., 138.],
         [103., 115.,  49.,  43.,  26., 344.]], dtype=torch.float64))

## OPENMAX

In [27]:
lower_stack=nn.Sequential(
            nn.Linear(1792, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, 64)
        )
# Initialize method
# epsilon = torchosr.utils.base.get_softmax_epsilon(OTHER_LABEL)
# model = TSoftmax(lower_stack=lower_stack, n_known=OTHER_LABEL, epsilon=epsilon)
epsilon = torchosr.utils.base.get_openmax_epsilon(OTHER_LABEL)
model = Openmax(lower_stack=lower_stack, n_known=OTHER_LABEL, epsilon=epsilon)
model.to(device)

writer = SummaryWriter()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

results = []
best_result = 0
for epoch in tqdm(range(50)):
    model.train(train_dataloader, loss_fn, optimizer)

    # Test
    model.to("cpu")
    result  = model.test(val_dataloader, loss_fn)[-1].item()
    model.to(device)

    results.append(result)
    writer.add_scalar("OSR/OpenMax", result, epoch)
    
    if(result>best_result):
        best_result = result
        torch.save(model.state_dict(),MULTIMODAL_MODEL_OPENMAX_PATH)

model = Openmax(lower_stack=lower_stack, n_known=OTHER_LABEL, epsilon=epsilon)
model.to("cpu")
model.load_state_dict(torch.load(MULTIMODAL_MODEL_OPENMAX_PATH))
print(model.test(val_dataloader, loss_fn)[-1].item(), best_result)


100%|██████████| 50/50 [01:44<00:00,  2.08s/it]


0.43961843848228455 0.4499533474445343


## T-SOFTMAX

In [28]:
lower_stack=nn.Sequential(
            nn.Linear(1792, 1024),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512, 64)
        )
# Initialize method
epsilon = torchosr.utils.base.get_softmax_epsilon(OTHER_LABEL)
model = TSoftmax(lower_stack=lower_stack, n_known=OTHER_LABEL, epsilon=epsilon)
model.to(device)

writer = SummaryWriter()

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

results = []
best_result = 0
for epoch in tqdm(range(50)):
    model.train(train_dataloader, loss_fn, optimizer)
    
    # Test
    model.to("cpu")
    result  = model.test(val_dataloader, loss_fn)[-1].item()
    model.to(device)

    results.append(result)
    writer.add_scalar("OSR/T-softmax", result, epoch)
    
    if(result>best_result):
        best_result = result
        torch.save(model.state_dict(),MULTIMODAL_MODEL_TSOFTMAX_PATH)

model = TSoftmax(lower_stack=lower_stack, n_known=OTHER_LABEL, epsilon=epsilon)
model.to("cpu")
model.load_state_dict(torch.load(MULTIMODAL_MODEL_TSOFTMAX_PATH))
print(model.test(val_dataloader, loss_fn)[-1].item(), best_result)


100%|██████████| 50/50 [01:03<00:00,  1.27s/it]


0.4311894476413727 0.4389110505580902


In [34]:
# learn pytorch basic with some basic models and datasets
# https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html
# https://pytorch.org/tutorials/beginner/basics/transforms_tutorial.html
# https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
# https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html
# https://pytorch.org/tutorials/beginner/basics/autogradqs_tutorial.html
# https://pytorch.org/tutorials/beginner/basics/nnqs_tutorial.html